In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from tqdm import tqdm
import csv
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import pandas as pd
import unidecode
from cmath import nan
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler , RobustScaler, StandardScaler
import warnings
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import os
from selenium_recaptcha import Recaptcha_Solver
warnings.filterwarnings("ignore")

    Crawl data

    URL = https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/3/da-nang.html

Khởi tạo đối tượng webdriver và khai báo link URL

In [2]:
#URL = https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/3/da-nang.html
productLinks = []

Lấy tất cả các link của bất động sản tại Đà Nẵng

In [3]:
urlDaNang = "https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/3/da-nang.html/trang--{}.html"
browser = webdriver.Chrome(executable_path="./chromedriver.exe")
for pagenum in range(1,1460):
   try:
      browser.get(urlDaNang.format(pagenum))
      if(len(browser.find_elements(By.CLASS_NAME, 'verify')) > 0):
         try:
            time.sleep(3)
            btn = browser.find_element(By.CLASS_NAME, 'verify')
            solver = Recaptcha_Solver(
               driver= browser, # Your Web Driver
               debug=False
            )
            solver.solve_recaptcha() 
            btn.click()
         except:
            print("no capcha")
            pass
      tagAs = browser.find_elements(By.LINK_TEXT, '<< xem chi tiết >>')
      for tagA in tagAs:
         print(tagA.get_attribute("href"))
         productLinks.append(tagA.get_attribute("href"))
   except:
      continue
browser.quit()
productLinks

https://alonhadat.com.vn/gia-soc-nha-sup-ham-300-trieu-nha-2-tang-thanh-luong-hoa-xuan-tp-da-nang-12728154.html
https://alonhadat.com.vn/ban-nha-3-tang-lo-goc-dong-nam-mat-me-trung-tam-hai-chau-nhinh-7-ty-12727729.html
https://alonhadat.com.vn/nha-2-tang-sat-bien-my-khe-khu-pho-tay-sam-uat-nhat-da-nang-12721269.html
https://alonhadat.com.vn/can-ban-gap-biet-thu-san-vuon-caphe-ngay-trung-tam-quan-lien-chieu--12635634.html
https://alonhadat.com.vn/chuyen-nhuong-can-goc-hoa-hau-biet-thu-regal-victoria-2-mat-tien-suat-ngoai-giao-cuoi-cu-11755800.html
https://alonhadat.com.vn/ban-nha-duong-bach-dang-vi-tri-vang-mat-song-han-pho-di-bo-170m2-gia-83-ty-12719579.html
https://alonhadat.com.vn/chi-voi-700tr-so-huu-ngay-can-ho-2pn-ven-song-cach-bien-chi-1-5km-12398677.html
https://alonhadat.com.vn/chinh-chu-can-ban-nha-3-tang-2-mat-kiet-va-dat-lien-ke-dia-chi-tran-cao-van-da-na-12659606.html
https://alonhadat.com.vn/ban-nha-mp-bach-dang-vip-nhat-da-nang-180m-hon-80-ty--12726097.html
https://alonha

Lấy các thuộc tính của bất động sản

In [ ]:
detailProductsAloDN = []
browser = webdriver.Chrome(executable_path="chromedriver")
for link in productLinks:
    try:
        browser.get(link)
        if(len(browser.find_elements(By.CLASS_NAME, 'verify')) > 0):
         try:
            time.sleep(3)
            btn = browser.find_element(By.CLASS_NAME, 'verify')
            solver = Recaptcha_Solver(
               driver= browser, # Your Web Driver
               debug=False
            )
            solver.solve_recaptcha() 
            btn.click()
         except:
            print("no capcha")
            pass
        data = browser.find_element(By.XPATH, "//div[@class='content plp']")
        try:
            name = data.find_element(By.TAG_NAME, "h1").text
            detailProduct = {"name": name}
        except:
            pass
        try:
            price = data.find_element(By.CLASS_NAME, "price").find_element(By.CLASS_NAME, "value").text
            detailProduct["Giá"] = price
        except:
            pass
        try:
            address = data.find_element(By.CLASS_NAME, "address").find_element(By.CLASS_NAME, "value").text
            detailProduct["Địa chỉ"] = address
        except:
            pass
        try:
            infor =  data.find_elements(By.TAG_NAME, "td")
            for i in range(0, len(infor), 2):
                detailProduct[infor[i].text] = infor[i + 1].text if(infor[i + 1].text != '') else 'true'
        except:
            pass
        detailProductsAloDN.append(detailProduct)
        print(detailProduct)
    except:
        print("link")
        continue
browser.quit()
detailProductsAloDN

{'name': 'Nhà 2 tầng sát biển Mỹ Khê khu phố Tây sầm uất nhất Đà Nẵng', 'Giá': '13,8 tỷ', 'Địa chỉ': '55 Đường Ngô Thì Sĩ, Phường Mỹ An, Quận Ngũ Hành Sơn, Đà Nẵng', 'Mã tin': '12721269', 'Hướng': '_', 'Phòng ăn': 'true', 'Loại tin': 'Cần bán', 'Đường trước nhà': '7,5m', 'Nhà bếp': 'true', 'Loại BDS': 'Nhà mặt tiền', 'Pháp lý': '---', 'Sân thượng': 'true', 'Chiều ngang': '5m', 'Số lầu': '2', 'Chổ để xe hơi': 'true', 'Chiều dài': '21m', 'Số phòng ngủ': '4', 'Chính chủ': '---'}
{'name': 'Cần bán gấp biệt thự sân vườn caphe ngay trung tâm quận liên chiểu .', 'Giá': '9,9 tỷ', 'Địa chỉ': '36-38 Đường Nguyễn Khuyến, Phường Hòa Minh, Quận Liên Chiểu, Đà Nẵng', 'Mã tin': '12635634', 'Hướng': 'Đông Bắc', 'Phòng ăn': 'true', 'Loại tin': 'Cần bán', 'Đường trước nhà': '7,5m', 'Nhà bếp': 'true', 'Loại BDS': 'Nhà mặt tiền', 'Pháp lý': '---', 'Sân thượng': 'true', 'Chiều ngang': '10m', 'Số lầu': '1', 'Chổ để xe hơi': 'true', 'Chiều dài': '20m', 'Số phòng ngủ': '2', 'Chính chủ': '---'}
{'name': 'Bán n

[{'name': 'Nhà 2 tầng sát biển Mỹ Khê khu phố Tây sầm uất nhất Đà Nẵng',
  'Giá': '13,8 tỷ',
  'Địa chỉ': '55 Đường Ngô Thì Sĩ, Phường Mỹ An, Quận Ngũ Hành Sơn, Đà Nẵng',
  'Mã tin': '12721269',
  'Hướng': '_',
  'Phòng ăn': 'true',
  'Loại tin': 'Cần bán',
  'Đường trước nhà': '7,5m',
  'Nhà bếp': 'true',
  'Loại BDS': 'Nhà mặt tiền',
  'Pháp lý': '---',
  'Sân thượng': 'true',
  'Chiều ngang': '5m',
  'Số lầu': '2',
  'Chổ để xe hơi': 'true',
  'Chiều dài': '21m',
  'Số phòng ngủ': '4',
  'Chính chủ': '---'},
 {'name': 'Cần bán gấp biệt thự sân vườn caphe ngay trung tâm quận liên chiểu .',
  'Giá': '9,9 tỷ',
  'Địa chỉ': '36-38 Đường Nguyễn Khuyến, Phường Hòa Minh, Quận Liên Chiểu, Đà Nẵng',
  'Mã tin': '12635634',
  'Hướng': 'Đông Bắc',
  'Phòng ăn': 'true',
  'Loại tin': 'Cần bán',
  'Đường trước nhà': '7,5m',
  'Nhà bếp': 'true',
  'Loại BDS': 'Nhà mặt tiền',
  'Pháp lý': '---',
  'Sân thượng': 'true',
  'Chiều ngang': '10m',
  'Số lầu': '1',
  'Chổ để xe hơi': 'true',
  'Chiều dà

Ghi dữ liệu qua file excel

In [ ]:
csv_columns = []
for product in detailProductsAloDN:
    tempkhac = set(list(product)[0:]) ^ set(csv_columns)
    tempgiong = set(list(product)[0:]) & set(csv_columns)
    csv_columns = tempkhac ^ tempgiong
csv_file = "dataBatDongSanDNAlo.csv"
try:
    with open(csv_file, 'w+',encoding= "utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for dataz in detailProductsAloDN:
            try:
                writer.writerow(dataz)
            except:
                continue
except IOError:
    print("I/O error")